In [11]:
import pandas as pd
import os
import numpy as np
from sklearn.datasets import fetch_20newsgroups

In [24]:
test_df.columns = ['text','type','summary','type_code']
test_df = test_df.reset_index(drop = True)
test_df.head()
test_df.to_csv('../data/temp/test.csv', index=False)

In [25]:
test_df.shape[0]

50

In [133]:
data_folder = "../data/raw/BBC News Summary/News Articles/"
entries = os.listdir(data_folder)
entries

['business', 'entertainment', 'politics', 'sport', 'tech']

In [131]:
all_data = {}
for i in entries:
    temp = []
    folder_path = data_folder + i +'/'
    file_lst = os.listdir(folder_path)
    for j in file_lst:
        if j != '.ipynb_checkpoints':
            with open(folder_path +j, 'r', errors='ignore') as file:
                temp.append(file.read().replace('\n', ''))
    all_data[i] = temp

In [ ]:
summary_folder = "../data/raw/BBC News Summary/Summaries/"
summary_entries = os.listdir(data_folder)
all_sum = {}
for i in entries:
    temp = []
    folder_path = summary_folder + i +'/'
    file_lst = os.listdir(folder_path)
    for j in file_lst:
        if j != '.ipynb_checkpoints':
            with open(folder_path +j, 'r', errors='ignore') as file:
                temp.append(file.read().replace('\n', ''))
    all_sum[i] = temp

In [21]:
total_df = pd.DataFrame()
test_df = pd.DataFrame()
for i in np.arange(len(entries)):
    if i == 0:
        total_df = pd.DataFrame.from_dict(all_data['business'])
        total_df['type'] = 'business'
        total_df['summary'] = all_sum['business']
        total_df['type_code'] = i+1
        
        
        index = np.random.choice(total_df.shape[0],10)
        test_df = total_df.loc[index]
    else:
        temp_df = pd.DataFrame.from_dict(all_data[entries[i]])
        temp_df['type'] = entries[i]
        temp_df['summary'] = all_sum[entries[i]]
        temp_df['type_code'] = i+1
        total_df =pd.concat([total_df, temp_df], axis=0)
        
        index = np.random.choice(temp_df.shape[0], 10)
        temp_test_df = temp_df.loc[index]
        test_df =pd.concat([test_df,temp_test_df] , axis=0) 
        
total_df.columns = ['text','type','summary','type_code']
total_df = total_df.reset_index(drop = True)
total_df.to_csv('../data/temp/all_data.csv', index=False)

In [22]:
total_df.shape[0]

2225

In [26]:
data_folder = "../data/raw/BBC News Summary/News Articles/"
entries = os.listdir('../data/raw/BBC News Summary/News Articles/')

In [32]:
temp = "../data/raw/BBC News Summary"

In [14]:
newsgroups_train = fetch_20newsgroups(subset='train')

In [159]:
news_df = pd.DataFrame.from_dict(newsgroups_train,'index').T

In [29]:
import re

In [199]:
def regex_condi(string):
    if ".com" in string:
        return False
    if "@" in string:
        return False
    return True
def clean_text(string):
    string = " ".join([i for i in string.split() if regex_condi(i)])
    string = re.sub(r"^From:.+ Lines: ","", string)
    string = re.sub(r"\'", "", string)
    string = re.sub(r"\>", "", string)
    string = re.sub(r"\:", "", string)
    string = re.sub(r"^[0-9]+ ", "", string)
    string = re.sub(r"\/", ' ',string)
    string = re.sub(r"\-{2,}", ' ',string)
    string = re.sub(r"\s+", ' ',string)
    string = string.strip()
    return string

In [200]:
clean_text(news_df.data[0]) 

'I was wondering if anyone out there could enlighten me on this car I saw the other day. It was a 2-door sports car, looked to be from the late 60s early 70s. It was called a Bricklin. The doors were really small. In addition, the front bumper was separate from the rest of the body. This is all I know. If anyone can tellme a model name, engine specs, years of production, where this car is made, history, or whatever info you have on this funky looking car, please e-mail. Thanks, - IL brought to you by your neighborhood Lerxst'

In [201]:
with open('../data/temp/text.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(news_df.data.apply(lambda x: clean_text(x)+ '.')))

In [134]:
data_folder = "../data/raw/20_newsgroups/"
entries = os.listdir(data_folder)
all_data = {}
for i in entries:
    temp = []
    folder_path = data_folder + i +'/'
    file_lst = os.listdir(folder_path)
    for j in file_lst:
        if j != '.ipynb_checkpoints':
            with open(folder_path +j, 'r', errors='ignore') as file:
                temp.append(file.read().replace('\n', ''))
    all_data[i] = temp

In [137]:
total_df = pd.DataFrame()
test_df = pd.DataFrame()
for i in np.arange(len(entries)):
    if i == 0:
        total_df = pd.DataFrame.from_dict(all_data['alt.atheism'])
        total_df['type'] = 'alt.atheism'
        total_df['type_code'] = i+1
        
        
        index = np.random.choice(total_df.shape[0],10)
        test_df = total_df.loc[index]
    else:
        temp_df = pd.DataFrame.from_dict(all_data[entries[i]])
        temp_df['type'] = entries[i]
        temp_df['type_code'] = i+1
        total_df =pd.concat([total_df, temp_df], axis=0)
        
        index = np.random.choice(temp_df.shape[0], 10)
        temp_test_df = temp_df.loc[index]
        test_df =pd.concat([test_df,temp_test_df] , axis=0) 
        

total_df = total_df.reset_index(drop = True)


In [ ]:
total_df.to_csv('../data/temp/all_data.csv', index=False)

In [110]:
path ="../data/temp/all_data.csv"

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer

In [13]:
df = pd.read_csv(path)
X = df.text
Y = df.type_code

NameError: name 'path' is not defined

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4)

NameError: name 'X' is not defined

In [142]:
X_val,X_test, y_val,y_test = train_test_split(X_test, y_test, test_size=0.5)

In [143]:
BoG = CountVectorizer().fit(X_train)

In [144]:
Bog_X_train = BoG.transform(X_train)
Bog_X_val = BoG.transform(X_val)
Bog_X_test = BoG.transform(X_test)

In [14]:
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression

In [9]:
clf = LogisticRegression(max_iter=500)
clf.fit(Bog_X_train,y_train)

NameError: name 'LogisticRegression' is not defined

In [151]:
predicted = clf.predict(Bog_X_train)
np.mean(predicted == y_train)

1.0

In [152]:
predicted = clf.predict(Bog_X_val)
np.mean(predicted == y_val)

0.952808988764045

In [153]:
predicted = clf.predict(Bog_X_test)
np.mean(predicted == y_test)

0.9640449438202248

In [10]:
X_train

NameError: name 'X_train' is not defined

In [164]:
tfidf = TfidfTransformer().fit(Bog_X_train)
tfidf_X = tfidf.transform(Bog_X_train)

In [165]:
clf = LogisticRegression(max_iter=500)
clf.fit(tfidf_X,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [166]:
predicted = clf.predict(Bog_X_train)
np.mean(predicted == y_train)

0.5325842696629214

In [167]:
predicted = clf.predict(Bog_X_val)
np.mean(predicted == y_val)

0.5123595505617977

In [168]:
predicted = clf.predict(Bog_X_test)
np.mean(predicted == y_test)

0.48089887640449436

In [1]:
from sklearn.datasets import fetch_20newsgroups

In [6]:
newsgroups_train = fetch_20newsgroups(subset='train')

In [7]:
newsgroups_test = fetch_20newsgroups(subset='test')

In [17]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(newsgroups_train.data)
count_vect.vocabulary_.get(u'algorithm')

27366

In [46]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([
     ('vect', CountVectorizer()),
     ('clf', LogisticRegression(max_iter = 100))
 ])
text_clf.fit(newsgroups_train.data, newsgroups_train.target)

C:\Users\Yang Li\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('clf',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='auto', n_jobs=None,
                          

In [49]:
import numpy as np
twenty_test = fetch_20newsgroups(subset='test')
docs_test = twenty_test.data
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)

0.7914232607541157

In [50]:
from sklearn import metrics
print(metrics.classification_report(twenty_test.target, predicted,target_names=twenty_test.target_names))

                          precision    recall  f1-score   support

             alt.atheism       0.73      0.73      0.73       319
           comp.graphics       0.68      0.75      0.71       389
 comp.os.ms-windows.misc       0.72      0.67      0.70       394
comp.sys.ibm.pc.hardware       0.66      0.69      0.67       392
   comp.sys.mac.hardware       0.76      0.80      0.78       385
          comp.windows.x       0.81      0.70      0.75       395
            misc.forsale       0.82      0.88      0.85       390
               rec.autos       0.83      0.85      0.84       396
         rec.motorcycles       0.90      0.90      0.90       398
      rec.sport.baseball       0.85      0.87      0.86       397
        rec.sport.hockey       0.93      0.92      0.93       399
               sci.crypt       0.89      0.87      0.88       396
         sci.electronics       0.69      0.72      0.70       393
                 sci.med       0.79      0.73      0.76       396
         

In [45]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(11314, 130107)

In [41]:
text_clf1 = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', LogisticRegression()),
])
text_clf1.fit(newsgroups_train.data, newsgroups_train.target)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
  

In [43]:
import numpy as np
twenty_test = fetch_20newsgroups(subset='test')
docs_test = twenty_test.data
predicted = text_clf1.predict(docs_test)
np.mean(predicted == twenty_test.target)

0.8274030801911842

In [44]:
from sklearn import metrics
print(metrics.classification_report(twenty_test.target, predicted,target_names=twenty_test.target_names))

                          precision    recall  f1-score   support

             alt.atheism       0.80      0.74      0.77       319
           comp.graphics       0.69      0.79      0.74       389
 comp.os.ms-windows.misc       0.75      0.73      0.74       394
comp.sys.ibm.pc.hardware       0.72      0.72      0.72       392
   comp.sys.mac.hardware       0.81      0.83      0.82       385
          comp.windows.x       0.83      0.74      0.78       395
            misc.forsale       0.76      0.90      0.82       390
               rec.autos       0.90      0.89      0.90       396
         rec.motorcycles       0.95      0.95      0.95       398
      rec.sport.baseball       0.88      0.92      0.90       397
        rec.sport.hockey       0.94      0.95      0.95       399
               sci.crypt       0.94      0.88      0.91       396
         sci.electronics       0.76      0.80      0.78       393
                 sci.med       0.89      0.83      0.85       396
         